## Set up our environment

In [ ]:
import sys
print(sys.executable)

In [ ]:
import pandas as pd
import numpy as np
from simple_salesforce import Salesforce
import math
from datetime import datetime
from datetime import timedelta 

## Decide if you want to view success messages when creating records

In [ ]:
log_info = False

## Load CRM Data into panda data frame

In [ ]:
#   create df from crm export
excel_file = r"C:\SSP Salesforce Load with Jupyter Template.xlsx"
crm_df = pd.read_excel(excel_file)

In [ ]:
# show list of columns in the spreadsheet
crm_df.columns = [c.lower().replace(' ', '_') for c in crm_df.columns]
crm_df.columns

In [ ]:
# accountList = crm_df.potential_customer.unique()
accountList = crm_df.account_name.unique()
len(accountList)

## Initialize the salesforce connection

In [ ]:
# info on the token: https://success.salesforce.com/answers?id=90630000000glADAAY
sfToken = 'enter_token_here'
sfUsername = 'enter_username_here'
sfPassword = 'enter_password_here'
sf = Salesforce(username=sfUsername, password=sfPassword, security_token=sfToken, client_id='Data Migration Application')

## Build out dict with ids of salesforce users

In [ ]:
# create the users manually in salesforce, a lot of decisions have to be made during that step so unless you
# have tons of users its easier to do that part using the salesforce interface

# build user dictionary (key: Name, value: Id)
userDict = sf.query_all("SELECT Id, Name FROM User")
user_dict = dict()
keys = iter(userDict.keys())
next(keys)
next(keys)
for key in keys:
    records = userDict[key]
    for record in records:
        user_dict[record['Name']] = record['Id']
user_dict

## A bunch of functions we'll use to parse the rows of data we're migrating

In [ ]:
def parse_string(value):
    if not isinstance(value, float):
        return value
    else:
        return None

In [ ]:
def parse_number(value):
    if math.isnan(value):
        return 0.0
    else:
        return value
    

In [ ]:
def get_active(value):
    if not isinstance(value, float) and value == 'Active':
        return True
    else:
        return False;

In [ ]:
def get_bool(value):
    if value == "Y":
        return True
    else:
        return False

In [ ]:
# get date from pandas timestamps
def parse_date(value):
    if not isinstance(value, float) and value is not pd.NaT:
        return value.date()
    else:
        return None

## Migrate accounts

In [ ]:
# loop through data frame and create new accounts while we create dict of new ids and account names
print("Migrating Accounts")
new_accounts = dict()
for new_account in crm_df.itertuples():
    name = getattr(new_account, "new_account_name_")  
    if log_info:
        print("Creating account {}...".format(name))
    typ = parse_string(getattr(new_account, "type"))
    ws = parse_string(getattr(new_account, "website"))
    desc  = parse_string(getattr(new_account, "description"))
    industry  = parse_string(getattr(new_account, "industry"))
    owner_id = ""
    master = getattr(new_account, "master_account_owner")
    try:
        owner_id = user_dict[master]
    except:
        print("User {} missing from salesforce. Skipping creation of account '{}'".format(master, name))
        continue
    new_obj = { "Name": name, "Type": typ, "Website": ws, "Description": desc, "Industry": industry, "OwnerId": owner_id }    
    res = sf.Account.create(new_obj)
    new_accounts[getattr(new_account, "account_name")] = res["id"]
    if log_info:
        print("Successfully created account {}...".format(name))
print("Finished")

## Load opportunites into data frame 

In [ ]:
opps = pd.read_excel(excel_file, sheet_name='Opportunities')
# show list of columns in the spreadsheet
opps.columns = [c.lower().replace(' ', '_') for c in opps.columns]
opps.columns

In [ ]:
sf = Salesforce(username=sfUsername, password=sfPassword, security_token=sfToken, client_id='Data Migration Application')

## Create opportunties

In [ ]:
iteration = 0
added_count = 0
for new_opp in opps.itertuples():
    iteration += 1
    if iteration % 500 == 0:
        print(iteration)
        sf = Salesforce(username=sfUsername, password=sfPassword, security_token=sfToken, client_id='Data Migration Application')
        
    opp_acct_name = getattr(new_opp, "account_name")
    name = getattr(new_opp, "opportunity_name")
    try:
        acct_id = new_accounts[opp_acct_name]
    except:
        print("Account '{}' missing. Skipping creation of opportunity '{}'...".format(opp_acct_name, name))
        continue
    try:
        owner = getattr(new_opp, "opportunity_owner")
        owner = owner.strip()
        owner_id = user_dict[owner]
    except:
        print("User {} missing from salesforce. Skipping creation of opportunity '{}'".format(owner, name))        
        continue
        
    stage = getattr(new_opp, "stage")
    prob = float(parse_number(getattr(new_opp, "probability")))
    
    close = parse_date(getattr(new_opp, "close_date"))            

    if close is None:
        print("Skipping creation of opportunity '{}' due to missing close date".format(name))
        continue

    close_str = close.isoformat()
            
    lead = parse_string(getattr(new_opp, "lead_source"))
    
    
    new_opp_obj = { "OwnerId": owner_id, "AccountId": acct_id, "Name": name,
                  "StageName": stage, "CloseDate": close_str,
                  "LeadSource": lead, }
    try:
        res = sf.Opportunity.create(new_opp_obj)        
        if log_info:
            print("Created opportunity '{}' for account '{}'...".format(name, opp_acct_name))        
    except Exception as e:
        print("Error creating opporunity '{}' for owner {}".format(name, owner))
        print (e)        
        break
    added_count += 1
        
print("Migrated {} opportunities".format(added_count))
    